In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import shapiro, ttest_ind, mannwhitneyu, chi2_contingency, fisher_exact
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
import matplotlib.pyplot as plt

In [ ]:
# Load the data 
df = pd.read_csv('sepsis_cleaned.csv')

In [ ]:
df.columns

In [ ]:
df['qsofa'] = df['qsofa'].apply(lambda x: 1 if x > 2 else 0)

In [ ]:
# Create some combined variables for later use
# create a new variable 'use of vasopressors'
df['use_of_vasopressors'] = df[['epinephrine', 'norepinephrine', 'dopamine', 'dobutamine']].sum(axis=1)
df['use_of_vasopressors'] = df['use_of_vasopressors'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
# create a new variable 'Invasive ventilation'
df['invasive_ventilation'] = df[['tracheostomy', 'IV']].sum(axis=1)
df['invasive_ventilation'] = df['invasive_ventilation'].apply(lambda x: 1 if x > 0 else 0)

# create a new variable 'Non invasive ventilation'
df['non_invasive_ventilation'] = df[['NIV', 'HFNC']].sum(axis=1)
df['non_invasive_ventilation'] = df['non_invasive_ventilation'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df['po2_fio2_geq'] = df['po2/fio2'].apply(lambda x: 1 if x > 450 else 0)
df['gcs_geq'] = df['gcs'].apply(lambda x: 1 if x > 14 else 0)

In [ ]:
# Set display options to show all columns without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df.tail()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
survivors = df[df['death'] == 0]
non_survivors = df[df['death'] == 1]

In [ ]:
survivors.shape

In [ ]:
non_survivors.shape

In [ ]:
categorical_variables = [
    'gender',
    'myocardial_infarction', 'congestive_heart_failure', 'peripheral_vascular_disease',
    'cerebrovascular_disease', 'dementia', 'chronic_pulmonary_disease',
    'rheumatic_disease', 'peptic_ulcer_disease', 'mild_liver_disease',
    'diabetes_without_chronic_complication', 'diabetes_with_chronic_complication',
    'hemiplegia_or_paraplegia', 'renal_disease', 'malignancy',
    'moderate_or_severe_liver_disease', 'metastatic_solid_tumor', 'AIDSHIV', 'hypertension',
    'diabetes_mellitus', 'epinephrine', 'norepinephrine', 'dopamine', 'dobutamine', 'tracheostomy',
    'IV', 'NIV', 'HFNC', 'supplemental_oxygen', 'gcs_geq', 'po2_fio2_geq', 'use_of_vasopressors',
    'invasive_ventilation', 'non_invasive_ventilation', 'qsofa'
]


In [ ]:
numeric_variables = [
    'BMI', 'temperature', 'heartrate', 'resprate', 'sbp', 'dbp',
    'o2sat',  'age',
    'WBC', 'platelet', 'log2_CRP', 'glucose', 'glucose_bg', 'lactate', 'creatinine', 'bilirubin'
]

In [ ]:
# Activate the automatic conversion from numpy to R arrays
numpy2ri.activate()

# Import the R "stats" package
stats = importr('stats')

def fisher_exact_test(table, workspace=1e6, simulate_p_value=True):
    result = stats.fisher_test(table, workspace=float(workspace), simulate_p_value=bool(simulate_p_value))
    p_value = result[0][0]
    return p_value

**Categorical variables** are expressed as the number and proportion, and compared by using the Χ2 test or Fisher’s exact test, as appropriate

In [ ]:
p_values = {} #store the p-values in a dictionary

In [ ]:
for variable in categorical_variables:
    contingency_table = pd.crosstab(df['death'], df[variable])
    stat, p, _, _ = chi2_contingency(contingency_table)
    
    if np.min(contingency_table.values) < 5:
        p = fisher_exact_test(contingency_table.values, workspace=1e6, simulate_p_value=True)
        print(f"Fisher's exact test p-value for {variable}: {p:.5f}")
    else:
        print(f"Chi-squared test p-value for {variable}: {p:.5f}")
    
    p_values[variable] = p

In [ ]:
# Filter predictor variables with p-value ≤ 0.05
significant_predictors = [var for var, p_value in p_values.items() if p_value <= 0.05]
print("\nSignificant predictor variables (p-value ≤ 0.05):")
print(significant_predictors)

In [ ]:
results_cat_df = pd.DataFrame()

# Loop through each categorical variable in your dataframe
for col in categorical_variables:
    
    # Calculate the total number and proportion of 1's for this variable
    total_count = df[col].value_counts().get(1, 0)
    total_prop = total_count / len(df)
    
    # Calculate the number and proportion of 1's for survivors
    survivor_count = df[df['death'] == 0][col].value_counts().get(1, 0)
    survivor_prop = survivor_count / len(df[df['death'] == 0])
    
    # Calculate the number and proportion of 1's for non-survivors
    nonsurvivor_count = df[df['death'] == 1][col].value_counts().get(1, 0)
    nonsurvivor_prop = nonsurvivor_count / len(df[df['death'] == 1])
    # Add the results to the new dataframe
    results_cat_df.loc[col, 'Total Count'] = total_count
    results_cat_df.loc[col, 'Total Proportion'] = total_prop*100
    results_cat_df.loc[col, 'Survivor Count'] = survivor_count
    results_cat_df.loc[col, 'Survivor Proportion'] = survivor_prop*100
    results_cat_df.loc[col, 'Non-Survivor Count'] = nonsurvivor_count
    results_cat_df.loc[col, 'Non-Survivor Proportion'] = nonsurvivor_prop*100
    results_cat_df.loc[col, 'P value'] = p_values[col]
    
# Convert integer columns to int type
int_cols = ['Total Count', 'Survivor Count', 'Non-Survivor Count']
results_cat_df[int_cols] = results_cat_df[int_cols].astype(int)

In [ ]:
# Print the results dataframe
results_cat_df.head()

**Continuous variables** were tested for normality using the Shapiro-Wilk test. Normally, distributed data are expressed as means and SDs, and compared between survivors and non-survivors by using Student’s t-test. Non-normally distributed data are expressed as medians and IQRs, and compared using the Mann-Whitney test.

In [ ]:
for col in numeric_variables:
    fig, ax = plt.subplots(figsize=(8, 6))
    # plot the histogram for overall
    df[col].hist(ax=ax, label='Overall')
    # plot the histogram for survivors
    df[df['death'] == 0][col].hist(ax=ax, alpha=0.7, label='Survivors')
    # plot the histogram for non-survivors
    df[df['death'] == 1][col].hist(ax=ax, alpha=0.7, label='Non-Survivors')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.legend()
    #fig.savefig(f'{col}_histogram.png')
    plt.show()


In [ ]:
results_num_df = pd.DataFrame()


def perform_statistical_tests(column):
    # Check for normality using the Shapiro-Wilk test
    stat, p_shapiro_survivors = shapiro(survivors[column])
    stat, p_shapiro_non_survivors = shapiro(non_survivors[column])
    
    if p_shapiro_survivors > 0.05 and p_shapiro_non_survivors > 0.05:
        # Normally distributed data: use Student's t-test
        stat, p = ttest_ind(survivors[column], non_survivors[column])
        print(f"Student's t-test p-value for {column}: {p:.5f}")
    else:
        # Non-normally distributed data: use Mann-Whitney test
        stat, p = mannwhitneyu(survivors[column], non_survivors[column])
        print(f"Mann-Whitney test p-value for {column}: {p:.5f}")
    p_values[column] = p

In [ ]:
for variable in numeric_variables:
    perform_statistical_tests(variable)

In [ ]:
# Filter predictor variables with p-value ≤ 0.05
significant_predictors = [var for var, p_value in p_values.items() if p_value <= 0.05]
print("\nSignificant predictor variables (p-value ≤ 0.05):")
print(significant_predictors)

In [ ]:

for col in numeric_variables:
    
    # Calculate the median and IQR for this variable among all patients
    overall_median = df[col].median()
    # Calculate the 25th and 75th percentiles of the 'age' column
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)

    # Calculate the IQR of the 'age' column
    overall_iqr = q3 - q1
    
    # Calculate the median and IQR for this variable among survivors (death=0)
    survivor_median = df[df['death'] == 0][col].median()
    survivor_q1 = df[df['death'] == 0][col].quantile(0.25)
    survivor_q3 = df[df['death'] == 0][col].quantile(0.75)
    survivor_iqr = survivor_q3 - survivor_q1
    
    
    # Calculate the number and proportion of 1's for non-survivors
    nonsurvivor_median = df[df['death'] == 1][col].median()
    nonsurvivor_q1 = df[df['death'] == 1][col].quantile(0.25)
    nonsurvivor_q3 = df[df['death'] == 1][col].quantile(0.75)
    nonsurvivor_iqr = survivor_q3 - survivor_q1
    
    
    # Add the results to the new dataframe
    results_num_df.loc[col, 'Overall Median'] = overall_median
    results_num_df.loc[col, 'Overall IQR'] = overall_iqr
    results_num_df.loc[col, 'Survivor Median'] = survivor_median
    results_num_df.loc[col, 'Survivor IQR'] = survivor_iqr
    results_num_df.loc[col, 'Non-Survivor Median'] = nonsurvivor_median
    results_num_df.loc[col, 'Non-Survivor IQR'] = nonsurvivor_iqr
    results_num_df.loc[col, 'P value'] = p_values[col]

In [ ]:
results_num_df.head(5)

In [ ]:
#Fix log2_CRP to CRP
col =  'log2_CRP'
results_num_df.loc[col, 'Overall Median'] = np.exp(overall_median * np.log(2))
results_num_df.loc[col, 'Overall IQR'] = np.exp(overall_iqr * np.log(2))
results_num_df.loc[col, 'Survivor Median'] = np.exp(survivor_median * np.log(2))
results_num_df.loc[col, 'Survivor IQR'] = np.exp(survivor_iqr * np.log(2))
results_num_df.loc[col, 'Non-Survivor Median'] = np.exp(nonsurvivor_median * np.log(2)) 
results_num_df.loc[col, 'Non-Survivor IQR'] = np.exp(nonsurvivor_iqr * np.log(2))
df.rename(columns={col: 'CRP'}, inplace=True)

Add missing value percents to each variable

In [ ]:
# define the file path and column names
file_path = 'sepsis.csv'

In [ ]:
cols = pd.DataFrame({'numbers': df.columns})


In [ ]:
df2 = pd.read_csv(file_path)
df2['use_of_vasopressors'] = df2[['epinephrine', 'norepinephrine', 'dopamine', 'dobutamine']].sum(axis=1, skipna=False)
df2['use_of_vasopressors'] = df2['use_of_vasopressors'].apply(lambda x: 1 if pd.notna(x) and x > 0 else (0 if pd.notna(x) and x == 0 else np.nan))

# create a new variable 'Invasive ventilation'
df2['invasive_ventilation'] = df2[['tracheostomy', 'IV']].sum(axis=1, skipna=False)
df2['invasive_ventilation'] = df2['invasive_ventilation'].apply(lambda x: 1 if pd.notna(x) and x > 0 else (0 if pd.notna(x) and x == 0 else np.nan))


# create a new variable 'Non invasive ventilation'
df2['non_invasive_ventilation'] = df2[['NIV', 'HFNC']].sum(axis=1, skipna=False)
df2['non_invasive_ventilation'] = df2['non_invasive_ventilation'].apply(lambda x: 1 if pd.notna(x) and x > 0 else (0 if pd.notna(x) and x == 0 else np.nan))

# create a new variable 'Non invasive ventilation'
df2['qsofa'] = df2[['resprate_first', 'gcs', 'sbp_first']].sum(axis=1, skipna=False)
df2['qsofa'] = df2['qsofa'].apply(lambda x: 1 if pd.notna(x) and x > 0 else (0 if pd.notna(x) and x == 0 else np.nan))


In [ ]:
# compute the missing value percentages for the categorical variables
cat_missing_perc = []
for var in categorical_variables:
    if var == 'po2_fio2_geq':
        var = 'po2/fio2'
    if var == 'gcs_geq':
        var = 'gcs'
    cat_missing_perc.append(df2[var].isna().mean())


In [ ]:
len(cat_missing_perc)

In [ ]:
results_cat_df['missing data'] = cat_missing_perc

In [ ]:
numeric_variables_modified = [
    'BMI', 'temperature_first', 'heartrate_first', 'resprate_first', 'sbp_first', 'dbp_first',
    'o2sat_first', 'age',
    'WBC_ED', 'platelet_ED', 'CRP', 'glucose_ED', 'glucose_bg_ED', 'lactate_ED', 'creatinine_ED', 'bilirubin_ED'
]

# compute the missing value percentages for the numeric variables
num_missing_perc = [df2[var].isna().mean() for var in numeric_variables_modified]

In [ ]:
num_missing_perc

In [ ]:
results_num_df['missing data'] = num_missing_perc

In [ ]:
 
writer = pd.ExcelWriter('results.xlsx')

# Write the results_num_df dataframe to the 'Numeric Results' sheet
results_num_df.to_excel(writer, sheet_name='Numeric Results')

# Write the results_cat_df dataframe to the 'Categorical Results' sheet
results_cat_df.to_excel(writer, sheet_name='Categorical Results')

# Save the Excel file
writer.save()

In [ ]:
import statsmodels.api as sm

In [ ]:
variables_sig = ['myocardial_infarction', 'cerebrovascular_disease', 
 'diabetes_with_chronic_complication', 'malignancy', 
                 'metastatic_solid_tumor',  
                 'invasive_ventilation', 'temperature', 'heartrate',
                  'qsofa', 'age']

In [ ]:
variables_domain = ['lactate' ,'diabetes_without_chronic_complication',
                    'moderate_or_severe_liver_disease', 'renal_disease',
                    'creatinine','use_of_vasopressors']

In [ ]:
df['EDTime'] = df['EDTime'] * 24

In [ ]:
# create new variables for each category of EDTime
df['EDTime_3'] = (df['EDTime'] < 3).astype(int)
df['EDTime_6'] = (df['EDTime'] < 6).astype(int)
df['EDTime_12'] = (df['EDTime'] < 12).astype(int)
df['EDTime_24'] = (df['EDTime'] < 24).astype(int)

In [ ]:
# create a list of all variables to include in the model
#variables = variables_sig + variables_domain + ['EDTime', 'WHITE', 'congestive_heart_failure']

In [ ]:
var = [ 'BMI', 'weight', 'age', 'myocardial_infarction',
       'congestive_heart_failure', 'peripheral_vascular_disease',
       'cerebrovascular_disease', 'dementia', 'chronic_pulmonary_disease',
       'rheumatic_disease', 'peptic_ulcer_disease', 'mild_liver_disease',
       'diabetes_without_chronic_complication',
       'diabetes_with_chronic_complication', 'hemiplegia_or_paraplegia',
       'renal_disease', 'malignancy', 'moderate_or_severe_liver_disease',
       'metastatic_solid_tumor', 'AIDSHIV', 'hypertension',
       'diabetes_mellitus', 'log2_num_admissions', 'log2_num_ED',
       'log2_num_ED_admissions', 'WBC', 'platelet', 'CRP', 'glucose',
       'glucose_bg', 'lactate', 'creatinine', 'bilirubin', 'po2_fio2_geq', 'gcs_geq',
       'epinephrine', 'norepinephrine', 'dopamine', 'dobutamine',
       'tracheostomy', 'IV', 'NIV', 'HFNC', 'supplemental_oxygen',
       'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'qsofa', 'use_of_vasopressors',
       'invasive_ventilation', 'non_invasive_ventilation'
       ]

In [ ]:
time_list = ['EDTime_3','EDTime_6', 'EDTime_12','EDTime_24']
or_dfs = []

In [ ]:
time_list = ['EDTime']

In [ ]:
from sklearn.preprocessing import StandardScaler


var_use = var + [time_range]

for time_range in time_list:
    
    X = df[var_use]
    scaler = StandardScaler()
    X_STD = scaler.fit_transform(X)
    # fit a logistic regression model with all variables
    X_STD = sm.add_constant(X_STD)
    y = df['death']
    logit_model = sm.Logit(y, X_STD)
    result = logit_model.fit(maxiter=100)
    
    # calculate adjusted odds ratios for each variable
    ORs = np.exp(result.params)
    CI = np.exp(result.conf_int(alpha=0.01))
    CI.columns = ['OR_lower', 'OR_upper']
    ORs = pd.concat([ORs, CI], axis=1)
    ORs.columns = ['OR', 'OR_lower', 'OR_upper']
    ORs.drop('const', inplace=True)
    #ORs = ORs.loc[var_use]
    or_dfs.append(ORs)
    

    print(ORs)

In [ ]:
var_use_df = pd.DataFrame(var_use, columns=['Variable'])


In [ ]:
filename = 'odds_ratios_final.xlsx'
with pd.ExcelWriter(filename) as writer:
    or_dfs[0].to_excel(writer, sheet_name='Sheet1', index=False)
    var_use_df.to_excel(writer, sheet_name='Sheet2', index=False)


In [ ]:
# Write the list of DataFrames to a single Excel file with separate sheets
with pd.ExcelWriter('odds_ratios_final.xlsx') as writer:
    for i, df_or in enumerate(or_dfs):
        df_or.to_excel(writer, sheet_name=time_list[i])

In [ ]:
or_file = pd.DataFrame(ORs)

# Save the DataFrame to an Excel file
or_file.to_excel('or_file_new_group.xlsx', index=True)

In [ ]:
# Create the plot
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.regplot(x='EDTime', y='death', data=df, logistic=True, ci=95, scatter=False, line_kws={'color': 'red', 'label': 'Fitted Line'})
#plt.fill_between(df_sorted['EDTime'], lower_conf_int[0], upper_conf_int[0], color='gray', alpha=0.3, label="95% CI")
plt.xlabel('Length of Stay in ED')
plt.ylabel('Adjusted Probability of Hospital Mortality')
plt.title('Adjusted Probability of Hospital Mortality vs. Length of Stay in ED')
plt.legend()
plt.show()